In [ ]:
library(tidyverse)
source("scripts/r/baypass_utils.R")
source("scripts/r/functions_objects.R")

In [ ]:
load_omega_mat <- function(path){
    mat <- read.table(path) %>% 
        as.matrix()
    return(list(mat = mat))
}

mat_list <- snakemake@input[["omega_mat"]] %>% 
    purrr::map(load_omega_mat)

In [ ]:
fmd_df <- utils::combn(1:length(mat_list), 2) %>% 
    t() %>% 
    as.data.frame() %>% 
    rename("n1" = "V1", "n2" = "V2")

calculate_fmd <- function(df){
    n1 <- df %>% pull(n1)
    n2 <- df %>% pull(n2)
    fmd <- fmd.dist(mat_list[[n1]]$mat, mat_list[[n2]]$mat)
    df_mod <- df %>% 
        mutate(fmd = fmd)
    return(df_mod)
}

fmd_df <- fmd_df %>% 
    group_split(n1, n2) %>% 
    purrr::map_dfr(., calculate_fmd)

In [ ]:
fmd_hist <- fmd_df %>% 
    ggplot(aes(x = fmd)) +
        geom_histogram(color = "black", fill = "grey") +
        xlab("Förstner and Moonen Distance (FMD)") +
        ylab("Number of comparisons") +
        my_theme

ggsave(filename = snakemake@output[["fmd_hist"]], plot = fmd_hist, 
       height = 8, width = 8, device = "pdf", dpi = 600, units = "in")